In [1]:
import pandas as pd
import requests
import json

In [2]:
matches_request = pd.read_csv('matches_request')

matches_request.head(5)

,Unnamed: 0,id,year,group,total_group,round,local,visitor,league_id,stadium,...,visitor_goals,result,live_minute,status,channels,winner,penaltis1,penaltis2,prorroga,stadium2
0,0,37429,2016,1,1,1,Málaga,Sevilla,15373,La Rosaleda,...,0,0-0,NaN,1,"[{'id': '41', 'name': 'C+ Liga', 'image': 'htt...",0,0,0,False,NaN
1,1,37433,2016,1,1,1,Deportivo,R. Sociedad,15373,Municipal Riazor,...,0,0-0,NaN,1,"[{'id': '41', 'name': 'C+ Liga', 'image': 'htt...",0,0,0,False,NaN
2,2,37432,2016,1,1,1,Espanyol,Getafe,15373,RCDE Stadium,...,0,1-0,NaN,1,"[{'id': '128', 'name': 'C+ Liga Multi', 'image...",214629,0,0,False,NaN
3,3,37437,2016,1,1,1,Atlético,Las Palmas,15373,Wanda Metropolitano,...,0,1-0,NaN,1,"[{'id': '41', 'name': 'C+ Liga', 'image': 'htt...",214622,0,0,False,NaN
4,4,37430,2016,1,1,1,Rayo Vallecano,Valencia,15373,Vallecas,...,0,0-0,NaN,1,"[{'id': '41', 'name': 'C+ Liga', 'image': 'htt...",0,0,0,False,NaN


In [3]:
matches_request.columns

Index(['Unnamed: 0', 'id', 'year', 'group', 'total_group', 'round', 'local',
       'visitor', 'league_id', 'stadium', 'team1', 'team2', 'conference',
       'dteam1', 'dteam2', 'numc', 'no_hour', 'local_abbr', 'visitor_abbr',
       'isVideo', 'numVideos', 'competition_name', 'competition_id',
       'split_league', 'type', 'type_id', 'playoffs', 'group_code',
       'total_rounds', 'coef', 'cflag_local', 'cflag_visitor', 'local_shield',
       'visitor_shield', 'extraTxt', 'schedule', 'date', 'hour', 'minute',
       'local_goals', 'visitor_goals', 'result', 'live_minute', 'status',
       'channels', 'winner', 'penaltis1', 'penaltis2', 'prorroga', 'stadium2'],
      dtype='object')

We drop out the columns that we evidently don´t need, and will re-evaluate later the remaining ones.

In [4]:
matches_merge = matches_request.drop(['Unnamed: 0','stadium', 'group', 'total_group', 'no_hour', 'status', 'isVideo', 'playoffs', 'penaltis1', 'penaltis2', 'prorroga', 'stadium2', 'channels','hour','minute', 'extraTxt', 'cflag_local', 'cflag_visitor','conference','local_shield', 'visitor_shield', 'schedule', 'total_rounds', 'type_id', 'type', 'split_league', 'live_minute', 'numVideos', 'no_hour', 'numc'] , axis = 1 )
matches_merge.columns

Index(['id', 'year', 'round', 'local', 'visitor', 'league_id', 'team1',
       'team2', 'dteam1', 'dteam2', 'local_abbr', 'visitor_abbr',
       'competition_name', 'competition_id', 'group_code', 'coef', 'date',
       'local_goals', 'visitor_goals', 'result', 'winner'],
      dtype='object')

We are not sure about the meaning of the columns 'dteam1' and 'team1'. They look like team identifyers

In [5]:
matches_merge['dteam1']

0       1617
1        901
2        998
3        369
4       2080
        ... 
5047    2115
5048    1578
5049     643
5050    2080
5051    2125
Name: dteam1, Length: 5052, dtype: int64

In [6]:
dteam_meaning = matches_merge['dteam1'] == 1617
dteam_meaning.head()

0     True
1    False
2    False
3    False
4    False
Name: dteam1, dtype: bool

In [7]:
dteam_1617 = matches_merge[dteam_meaning]
dteam_1617.tail()

,id,year,round,local,visitor,league_id,team1,team2,dteam1,dteam2,...,visitor_abbr,competition_name,competition_id,group_code,coef,date,local_goals,visitor_goals,result,winner
5001,91106,2021,38,Málaga,Mallorca,57314,6382794,6382795,1617,1623,...,MLL,Segunda División,2,1,58.462,2021/05/08,1,1,1-1,0
5012,91106,2021,38,Málaga,Mallorca,57314,6382794,6382795,1617,1623,...,MLL,Segunda División,2,1,58.462,2021/05/08,1,1,1-1,0
5023,91106,2021,38,Málaga,Mallorca,57314,6382794,6382795,1617,1623,...,MLL,Segunda División,2,1,58.462,2021/05/08,1,1,1-1,0
5034,91106,2021,38,Málaga,Mallorca,57314,6382794,6382795,1617,1623,...,MLL,Segunda División,2,1,58.462,2021/05/08,1,1,1-1,0
5045,91106,2021,38,Málaga,Mallorca,57314,6382794,6382795,1617,1623,...,MLL,Segunda División,2,1,58.462,2021/05/08,1,1,1-1,0


We realise that 'dteam' columns are identifyers for teams, but also 'team1' and 'team 2'. Researching into API documentation we find out the difference, 'team' is an identifyer of the team only for that season and  'dteam' is the unique team identifyer for all the seasons and competitions.


#### For any other user doubts, here you can check the documentation with the meanings of each column: https://company.besoccer.com/api/documentacion/matchs

In [8]:
standings_request = pd.read_csv('standings_request')
standings_request

,Unnamed: 0,id,group,group_name,conference,team,points,wins,draws,losses,...,coef,coefficients,mark,class_mark,round,pos,countrycode,abbr,form,direction
0,0,957,1,NaN,0,Eibar,3,1,0,0,...,NaN,NaN,1.0,cha,1,1,ES,EIB,w,NaN
1,1,712,1,NaN,0,Celta,3,1,0,0,...,NaN,NaN,1.0,cha,1,2,ES,CEL,w,NaN
2,2,369,1,NaN,0,Atlético,3,1,0,0,...,NaN,NaN,1.0,cha,1,3,ES,ATM,w,NaN
3,3,429,1,NaN,0,Barcelona,3,1,0,0,...,NaN,NaN,2.0,prev,1,4,ES,FCB,w,NaN
4,4,998,1,NaN,0,Espanyol,3,1,0,0,...,NaN,NaN,3.0,uefa,1,5,ES,ESP,w,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9571,17,673,1,NaN,0,CD Castellón,41,11,8,19,...,NaN,NaN,NaN,NaN,38,18,ES,CAS,lwddl,NaN
9572,18,1578,1,NaN,0,UD Logroñés,41,10,11,17,...,NaN,NaN,3.0,desc,38,19,ES,UDL,wdldl,d
9573,19,2198,1,NaN,0,Sabadell,40,9,13,16,...,NaN,NaN,3.0,desc,38,20,ES,SAB,ddwwl,d
9574,20,1598,1,NaN,0,Lugo,37,8,13,17,...,NaN,NaN,3.0,desc,38,21,ES,LUG,llldl,NaN


In [9]:
standings_request.columns

Index(['Unnamed: 0', 'id', 'group', 'group_name', 'conference', 'team',
       'points', 'wins', 'draws', 'losses', 'shield', 'cflag', 'basealias',
       'gf', 'ga', 'avg', 'matchs_coef', 'points_coef', 'coef', 'coefficients',
       'mark', 'class_mark', 'round', 'pos', 'countrycode', 'abbr', 'form',
       'direction'],
      dtype='object')

In [10]:
standings_request = standings_request.drop(['group', 'group_name', 'conference', 'shield', 'cflag', 'basealias',
       'matchs_coef', 'points_coef', 'coef', 'coefficients', 'mark',
       'class_mark', 'countrycode', 'abbr',
       'direction'] , axis = 1 )

In [11]:
standings_request.tail(3)

,Unnamed: 0,id,team,points,wins,draws,losses,gf,ga,avg,round,pos,form
9573,19,2198,Sabadell,40,9,13,16,36,44,-8,38,20,ddwwl
9574,20,1598,Lugo,37,8,13,17,32,50,-18,38,21,llldl
9575,21,140,Albacete,36,9,9,20,25,46,-21,38,22,lwdlw


There is some information left in this request that we need to be able to do de merge with the DataFrame created by Matches request. This time we increment directly the number of rounds, taking into account that LaLiga Smartbank has more teams.

In [12]:
for league_division in range(1, 3):
    for league_season in range(2016, 2022):
        for round_num in range(1, 43):
            standings_url_bis = f"https://apiclient.besoccerapps.com/scripts/api/api.php?key=023afbc77c5610fefc3fc8976e451752&tz=Europe/Madrid&format=json&req=tables&league={league_division}&round={round_num}&year={league_season}"
            response_standings_bis = requests.get(standings_url_bis)
            result_standings_bis = json.loads(response_standings_bis.content)
            if league_division == 1 and league_season == 2016 and round_num == 1:
                df4 = pd.DataFrame(result_standings_bis['table'])
                df4['year'] = league_season
                df4['division'] = league_division
            else:
                df4_bis = pd.DataFrame(result_standings_bis['table'])
                df4_bis['year'] = league_season
                df4_bis['division'] = league_division
                df4 = pd.concat([df4, df4_bis])

In [13]:
df4.head(5)

,id,group,group_name,conference,team,points,wins,draws,losses,shield,...,mark,class_mark,round,pos,countrycode,abbr,form,direction,year,division
0,957,1,,0,Eibar,3,1.0,0.0,0.0,https://thumb.resfu.com/img_data/escudos/mediu...,...,1,cha,1,1,ES,EIB,w,,2016,1
1,712,1,,0,Celta,3,1.0,0.0,0.0,https://thumb.resfu.com/img_data/escudos/mediu...,...,1,cha,1,2,ES,CEL,w,,2016,1
2,369,1,,0,Atlético,3,1.0,0.0,0.0,https://thumb.resfu.com/img_data/escudos/mediu...,...,1,cha,1,3,ES,ATM,w,,2016,1
3,429,1,,0,Barcelona,3,1.0,0.0,0.0,https://thumb.resfu.com/img_data/escudos/mediu...,...,2,prev,1,4,ES,FCB,w,,2016,1
4,998,1,,0,Espanyol,3,1.0,0.0,0.0,https://thumb.resfu.com/img_data/escudos/mediu...,...,3,uefa,1,5,ES,ESP,w,,2016,1


In [14]:
df4.columns

Index(['id', 'group', 'group_name', 'conference', 'team', 'points', 'wins',
       'draws', 'losses', 'shield', 'cflag', 'basealias', 'gf', 'ga', 'avg',
       'matchs_coef', 'points_coef', 'coef', 'coefficients', 'mark',
       'class_mark', 'round', 'pos', 'countrycode', 'abbr', 'form',
       'direction', 'year', 'division'],
      dtype='object')

In [15]:
standings_merge = df4
standings_merge.tail(4)

,id,group,group_name,conference,team,points,wins,draws,losses,shield,...,mark,class_mark,round,pos,countrycode,abbr,form,direction,year,division
18,2198,1,,0,Sabadell,46,11.0,13.0,18.0,https://thumb.resfu.com/img_data/escudos/mediu...,...,3,desc,42,19,ES,SAB,lllww,u,2021,2
19,1578,1,,0,UD Logroñés,44,11.0,11.0,20.0,https://thumb.resfu.com/img_data/escudos/mediu...,...,3,desc,42,20,ES,UDL,llwll,d,2021,2
20,673,1,,0,CD Castellón,41,11.0,8.0,23.0,https://thumb.resfu.com/img_data/escudos/mediu...,...,3,desc,42,21,ES,CAS,lllll,,2021,2
21,140,1,,0,Albacete,38,9.0,11.0,22.0,https://thumb.resfu.com/img_data/escudos/mediu...,...,3,desc,42,22,ES,ALB,wddll,,2021,2


We now clean and prepare to merge both dataframes, we need to create a key in common for both datasets, and apply it twice, once for local team and once for visitor.

In [16]:
standings_merge = standings_merge.drop(['group', 'group_name', 'conference', 'shield', 'cflag', 'basealias',
       'matchs_coef', 'points_coef', 'coef', 'coefficients', 'mark',
       'class_mark', 'countrycode', 'abbr',
       'direction'] , axis = 1 )

In [17]:
standings_merge.tail()

,id,team,points,wins,draws,losses,gf,ga,avg,round,pos,form,year,division
17,1598,Lugo,47,11.0,14.0,17.0,38,53,-15.0,42,18,lwdww,2021,2
18,2198,Sabadell,46,11.0,13.0,18.0,40,48,-8.0,42,19,lllww,2021,2
19,1578,UD Logroñés,44,11.0,11.0,20.0,28,53,-25.0,42,20,llwll,2021,2
20,673,CD Castellón,41,11.0,8.0,23.0,35,54,-19.0,42,21,lllll,2021,2
21,140,Albacete,38,9.0,11.0,22.0,30,53,-23.0,42,22,wddll,2021,2


In [18]:
standings_merge['key'] = standings_merge['id'].astype(str) + "_" + standings_merge['round'].apply(lambda x: int(x) + 1).astype(str) + "_" + standings_merge['year'].astype(str) + "_" + standings_merge['division'].astype(str)
standings_merge['key'].head(5)

0    957_2_2016_1
1    712_2_2016_1
2    369_2_2016_1
3    429_2_2016_1
4    998_2_2016_1
Name: key, dtype: object

While creating the key in the Standings dataframe it is important to understand why we add a +1 to the round number. The team position we find here corresponds to the position that team will have at the end of the round that appears in the corresponding row, that means, knowing the result of the match from that round. That is the reason why we have to merge each round position in this dataframe with one round after from the matches results one.

In [19]:
standings_merge.head(5)

,id,team,points,wins,draws,losses,gf,ga,avg,round,pos,form,year,division,key
0,957,Eibar,3,1.0,0.0,0.0,3,1,2.0,1,1,w,2016,1,957_2_2016_1
1,712,Celta,3,1.0,0.0,0.0,2,1,1.0,1,2,w,2016,1,712_2_2016_1
2,369,Atlético,3,1.0,0.0,0.0,1,0,1.0,1,3,w,2016,1,369_2_2016_1
3,429,Barcelona,3,1.0,0.0,0.0,1,0,1.0,1,4,w,2016,1,429_2_2016_1
4,998,Espanyol,3,1.0,0.0,0.0,1,0,1.0,1,5,w,2016,1,998_2_2016_1


In [20]:
matches_merge.head(3)

,id,year,round,local,visitor,league_id,team1,team2,dteam1,dteam2,...,visitor_abbr,competition_name,competition_id,group_code,coef,date,local_goals,visitor_goals,result,winner
0,37429,2016,1,Málaga,Sevilla,15373,214628,214624,1617,1102,...,SEV,Primera División,1,1,76.0,2015/08/21,0,0,0-0,0
1,37433,2016,1,Deportivo,R. Sociedad,15373,214634,214631,901,2120,...,RSO,Primera División,1,1,76.0,2015/08/22,0,0,0-0,0
2,37432,2016,1,Espanyol,Getafe,15373,214629,214633,998,1217,...,GET,Primera División,1,1,76.0,2015/08/22,1,0,1-0,214629


In [21]:
matches_merge.columns

Index(['id', 'year', 'round', 'local', 'visitor', 'league_id', 'team1',
       'team2', 'dteam1', 'dteam2', 'local_abbr', 'visitor_abbr',
       'competition_name', 'competition_id', 'group_code', 'coef', 'date',
       'local_goals', 'visitor_goals', 'result', 'winner'],
      dtype='object')

In [22]:
matches_merge = matches_merge.rename(columns={'id':'match_id', 'team1':'team1_id_season', 'team2':'team2_id_season','dteam1':'team1_id', 'dteam2':'team2_id', 'competition_id':'division'})

In [23]:
matches_merge.columns

Index(['match_id', 'year', 'round', 'local', 'visitor', 'league_id',
       'team1_id_season', 'team2_id_season', 'team1_id', 'team2_id',
       'local_abbr', 'visitor_abbr', 'competition_name', 'division',
       'group_code', 'coef', 'date', 'local_goals', 'visitor_goals', 'result',
       'winner'],
      dtype='object')

In [24]:
matches_merge = matches_merge.drop(['group_code', 'coef', 'date', 'competition_name'], axis = 1)
matches_merge.tail(5)

,match_id,year,round,local,visitor,league_id,team1_id_season,team2_id_season,team1_id,team2_id,local_abbr,visitor_abbr,division,local_goals,visitor_goals,result,winner
5047,91110,2021,38,Real Oviedo,Sabadell,57314,6382799,6382802,2115,2198,ROV,SAB,2,2,1,2-1,6382799
5048,91112,2021,38,UD Logroñés,Girona,57314,6382792,6391868,1578,1236,UDL,GIR,2,1,4,1-4,6391868
5049,91104,2021,38,FC Cartagena,CD Castellón,57314,6382787,6382788,643,673,CAR,CAS,2,1,0,1-0,6382787
5050,91109,2021,38,Rayo Vallecano,Leganés,57314,6382798,6382791,2080,1535,RAY,LEG,2,1,1,1-1,0
5051,91105,2021,38,Real Sporting,Lugo,57314,6382800,6382793,2125,1598,SPO,LUG,2,1,0,1-0,6382800


In [25]:
matches_merge['key_local'] = matches_merge['team1_id'].astype(str) + "_" + matches_merge['round'].astype(str) + "_" + matches_merge['year'].astype(str) + "_" + matches_merge['division'].astype(str)
matches_merge['key_visitor'] = matches_merge['team2_id'].astype(str) + "_" + matches_merge['round'].astype(str) + "_" + matches_merge['year'].astype(str) + "_" + matches_merge['division'].astype(str)

In [26]:
matches_merge.head()

,match_id,year,round,local,visitor,league_id,team1_id_season,team2_id_season,team1_id,team2_id,local_abbr,visitor_abbr,division,local_goals,visitor_goals,result,winner,key_local,key_visitor
0,37429,2016,1,Málaga,Sevilla,15373,214628,214624,1617,1102,MAL,SEV,1,0,0,0-0,0,1617_1_2016_1,1102_1_2016_1
1,37433,2016,1,Deportivo,R. Sociedad,15373,214634,214631,901,2120,DEP,RSO,1,0,0,0-0,0,901_1_2016_1,2120_1_2016_1
2,37432,2016,1,Espanyol,Getafe,15373,214629,214633,998,1217,ESP,GET,1,1,0,1-0,214629,998_1_2016_1,1217_1_2016_1
3,37437,2016,1,Atlético,Las Palmas,15373,214622,218388,369,2563,ATM,UDL,1,1,0,1-0,214622,369_1_2016_1,2563_1_2016_1
4,37430,2016,1,Rayo Vallecano,Valencia,15373,214630,214623,2080,2647,RAY,VCF,1,0,0,0-0,0,2080_1_2016_1,2647_1_2016_1


In [27]:
merged_data = matches_merge.merge(standings_merge, how = 'left', left_on = 'key_local', right_on = 'key' )
merged_data.head()

,match_id,year_x,round_x,local,visitor,league_id,team1_id_season,team2_id_season,team1_id,team2_id,...,losses,gf,ga,avg,round_y,pos,form,year_y,division_y,key
0,37429,2016,1,Málaga,Sevilla,15373,214628,214624,1617,1102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,37433,2016,1,Deportivo,R. Sociedad,15373,214634,214631,901,2120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37432,2016,1,Espanyol,Getafe,15373,214629,214633,998,1217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,37437,2016,1,Atlético,Las Palmas,15373,214622,218388,369,2563,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,37430,2016,1,Rayo Vallecano,Valencia,15373,214630,214623,2080,2647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As we can see, after the merge, in the head rounds we find out that is filled with Nans values. Is reasonable that this happens because of the key we created, every match corresponding to rounds = 1 will have the same problem. In any case, we check it with rest of the DataFrame and drop this values. 

In [28]:
merged_data


,match_id,year_x,round_x,local,visitor,league_id,team1_id_season,team2_id_season,team1_id,team2_id,...,losses,gf,ga,avg,round_y,pos,form,year_y,division_y,key
0,37429,2016,1,Málaga,Sevilla,15373,214628,214624,1617,1102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,37433,2016,1,Deportivo,R. Sociedad,15373,214634,214631,901,2120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37432,2016,1,Espanyol,Getafe,15373,214629,214633,998,1217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,37437,2016,1,Atlético,Las Palmas,15373,214622,218388,369,2563,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,37430,2016,1,Rayo Vallecano,Valencia,15373,214630,214623,2080,2647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5047,91110,2021,38,Real Oviedo,Sabadell,57314,6382799,6382802,2115,2198,...,11.0,38,38,0.0,37,14,ldwld,2021.0,2.0,2115_38_2021_2
5048,91112,2021,38,UD Logroñés,Girona,57314,6382792,6391868,1578,1236,...,16.0,25,43,-18.0,37,17,wwdld,2021.0,2.0,1578_38_2021_2
5049,91104,2021,38,FC Cartagena,CD Castellón,57314,6382787,6382788,643,673,...,16.0,36,47,-11.0,37,20,wddwl,2021.0,2.0,643_38_2021_2
5050,91109,2021,38,Rayo Vallecano,Leganés,57314,6382798,6382791,2080,1535,...,12.0,43,36,7.0,37,7,wlddl,2021.0,2.0,2080_38_2021_2


In [29]:
merged_data = merged_data.dropna()
merged_data

,match_id,year_x,round_x,local,visitor,league_id,team1_id_season,team2_id_season,team1_id,team2_id,...,losses,gf,ga,avg,round_y,pos,form,year_y,division_y,key
10,37467,2016,2,Villarreal,Espanyol,15373,214625,214629,2716,998,...,0.0,1,1,0.0,1,7,d,2016.0,1.0,2716_2_2016_1
11,37471,2016,2,R. Sociedad,Real Sporting,15373,214631,214619,2120,2125,...,0.0,0,0,0.0,1,10,d,2016.0,1.0,2120_2_2016_1
12,37463,2016,2,Barcelona,Málaga,15373,214620,214628,429,1617,...,0.0,1,0,1.0,1,4,w,2016.0,1.0,429_2_2016_1
13,37457,2016,2,Celta,Rayo Vallecano,15373,214627,214630,712,2080,...,0.0,2,1,1.0,1,2,w,2016.0,1.0,712_2_2016_1
14,37469,2016,2,Real Madrid,Real Betis,15373,214621,214618,2107,486,...,0.0,0,0,0.0,1,12,d,2016.0,1.0,2107_2_2016_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5047,91110,2021,38,Real Oviedo,Sabadell,57314,6382799,6382802,2115,2198,...,11.0,38,38,0.0,37,14,ldwld,2021.0,2.0,2115_38_2021_2
5048,91112,2021,38,UD Logroñés,Girona,57314,6382792,6391868,1578,1236,...,16.0,25,43,-18.0,37,17,wwdld,2021.0,2.0,1578_38_2021_2
5049,91104,2021,38,FC Cartagena,CD Castellón,57314,6382787,6382788,643,673,...,16.0,36,47,-11.0,37,20,wddwl,2021.0,2.0,643_38_2021_2
5050,91109,2021,38,Rayo Vallecano,Leganés,57314,6382798,6382791,2080,1535,...,12.0,43,36,7.0,37,7,wlddl,2021.0,2.0,2080_38_2021_2


In [30]:
merged_data.columns


Index(['match_id', 'year_x', 'round_x', 'local', 'visitor', 'league_id',
       'team1_id_season', 'team2_id_season', 'team1_id', 'team2_id',
       'local_abbr', 'visitor_abbr', 'division_x', 'local_goals',
       'visitor_goals', 'result', 'winner', 'key_local', 'key_visitor', 'id',
       'team', 'points', 'wins', 'draws', 'losses', 'gf', 'ga', 'avg',
       'round_y', 'pos', 'form', 'year_y', 'division_y', 'key'],
      dtype='object')

In [31]:
merged_data.to_csv('merged_data')